In [0]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Sun Jun  7 07:49:23 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.82       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   31C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install transformers==2.10.0

     |███████                         | 143kB 3.5MB/s eta 0:00:01
ERROR: Operation cancelled by user


In [0]:
!pip install simpletransformers

In [2]:
!pip install wandb

In [3]:
!wandb login

wandb: You can find your API key in your browser here: https://app.wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter: 466ed0b4b2a230df648b8b1636be583c783c283e
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
Successfully logged in to Weights & Biases!


In [1]:
cd "/content/drive/My Drive/Colab Notebooks/binary_classifier"

/content/drive/My Drive/Colab Notebooks/binary_classifier


In [0]:
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from simpletransformers.classification import ClassificationModel
import logging
import sklearn



1. Input Data

In [3]:
prefix = 'data/'

raw_data = pd.read_excel(prefix + "positive_data.xlsx")
pos_data = raw_data[['Sentence']].drop_duplicates("Sentence")
pos_data['Label'] = 1

neg_data = pd.read_excel(prefix + "negative_data.xlsx")
neg_data['Label'] = 0

dataframe = pd.concat([pos_data[['Sentence', 'Label']], neg_data[['Sentence', 'Label']]])
dataframe = dataframe.rename(columns={"Sentence": "text", "Label": "labels"})
dataframe = dataframe.reset_index(drop=True)

print(dataframe.groupby('labels').count())
print(dataframe.head())

        text
labels      
0        239
1        303
                                                text  labels
0  Stopping mass gatherings is predicted to have ...       1
1  Adding household quarantine to isolation of th...       1
2  Preventing mass gatherings is expected to have...       1
3  At the same time, by implementing population-w...       1
4  As evidence suggests COVID-19 could be transmi...       1


2. Prepare the Training and Testing Data sets

In [4]:
# split the dataset into training and testing datasets 
train_df, eval_df = train_test_split(dataframe, 
                                     random_state=1, 
                                     test_size=0.3, 
                                     shuffle=True,
                                     stratify = dataframe['labels'])

train_df = train_df.reset_index(drop=True)
print(train_df.groupby('labels').count())
print(train_df.head())

eval_df = eval_df.reset_index(drop=True)
print(eval_df.groupby('labels').count())
print(eval_df.head())

        text
labels      
0        167
1        212
                                                text  labels
0  Internal travel restrictions in England, Scotl...       1
1  The closure of public places and restrictions ...       1
2  Our goal is to experimentally demonstrate the ...       0
3  Quarantine at home can play an important role ...       1
4  This experiment could help us understand the r...       0
        text
labels      
0         72
1         91
                                                text  labels
0      Will wearing masks help control the epidemic?       0
1  During the 1918–19 pandemic, excess death rate...       1
2  Given that influenza patients can already tran...       1
3  In controlled trials using hand sanitizers and...       1
4  Travel bans and home quarantine are equally im...       1


3. Set the hyperparameters of model

In [0]:
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

train_args={
    'fp16': False,
    'sliding_window': True,
    'reprocess_input_data': True,
    'overwrite_output_dir': True,
    
    'evaluate_during_training': True,
    'evaluate_during_training_steps': 4,
    'logging_steps': 4,
    'evaluate_during_training_verbose': True,

    'num_train_epochs': 5,
    
    'learning_rate': 3.5e-5,
    
    'train_batch_size': 32,
    'eval_batch_size': 8,
    
    'warmup_ratio': 0.1, 
    'warmup_steps': 14,

    'weight_decay': 0.01,
    
    'wandb_project': "NPI_binary",
    
    'save_eval_checkpoints': False,
    "save_model_every_epoch": False,
    'use_cached_eval_features': True,
    
    'manual_seed': 1
}



4. Train the model using BERT

In [8]:
# Create a TransformerModel
model = ClassificationModel('roberta', 
                            'roberta-large',
                            args=train_args
                           )

# Train the model
model.train_model(train_df, 
                  eval_df=eval_df,
                  acc = sklearn.metrics.accuracy_score,
                  f1_score = sklearn.metrics.f1_score
                 ) 


INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.
INFO:simpletransformers.classification.classification_model: Sliding window enabled


INFO:simpletransformers.classification.classification_model: 379 features created from 379 samples.


INFO:wandb.run_manager:system metrics and metadata threads started
INFO:wandb.run_manager:checking resume status, waiting at most 10 seconds
INFO:wandb.run_manager:resuming run from id: UnVuOnYxOjFyMWhxcWR0Ok5QSV9iaW5hcnk6cnlhbmxlZQ==
INFO:wandb.run_manager:upserting run before process can begin, waiting at most 10 seconds
INFO:wandb.run_manager:saving pip packages
INFO:wandb.run_manager:initializing streaming files api
INFO:wandb.run_manager:unblocking file change observer, beginning sync with W&B servers


INFO:wandb.run_manager:file/dir created: /content/drive/My Drive/Colab Notebooks/binary_classifier/wandb/run-20200608_152655-1r1hqqdt/wandb-events.jsonl
INFO:wandb.run_manager:file/dir created: /content/drive/My Drive/Colab Notebooks/binary_classifier/wandb/run-20200608_152655-1r1hqqdt/wandb-history.jsonl
INFO:wandb.run_manager:file/dir created: /content/drive/My Drive/Colab Notebooks/binary_classifier/wandb/run-20200608_152655-1r1hqqdt/wandb-metadata.json
INFO:wandb.run_manager:file/dir created: /content/drive/My Drive/Colab Notebooks/binary_classifier/wandb/run-20200608_152655-1r1hqqdt/wandb-summary.json
INFO:wandb.run_manager:file/dir created: /content/drive/My Drive/Colab Notebooks/binary_classifier/wandb/run-20200608_152655-1r1hqqdt/requirements.txt


Running loss: 0.720506

...

INFO:wandb.run_manager:file/dir modified: /content/drive/My Drive/Colab Notebooks/binary_classifier/wandb/run-20200608_152655-1r1hqqdt/wandb-history.jsonl
INFO:wandb.run_manager:file/dir modified: /content/drive/My Drive/Colab Notebooks/binary_classifier/wandb/run-20200608_152655-1r1hqqdt/wandb-summary.json


Running loss: 0.000431

INFO:simpletransformers.classification.classification_model: Features loaded from cache at cache_dir/cached_dev_roberta_128_2_163
INFO:wandb.run_manager:file/dir modified: /content/drive/My Drive/Colab Notebooks/binary_classifier/wandb/run-20200608_152655-1r1hqqdt/wandb-history.jsonl
INFO:wandb.run_manager:file/dir modified: /content/drive/My Drive/Colab Notebooks/binary_classifier/wandb/run-20200608_152655-1r1hqqdt/wandb-summary.json
INFO:simpletransformers.classification.classification_model:{'mcc': 0.9128717482439989, 'tp': 88, 'tn': 68, 'fp': 4, 'fn': 3, 'acc': 0.9570552147239264, 'f1_score': 0.9617486338797815, 'eval_loss': 0.22090506364613594}
INFO:wandb.run_manager:file/dir modified: /content/drive/My Drive/Colab Notebooks/binary_classifier/wandb/run-20200608_152655-1r1hqqdt/wandb-events.jsonl
INFO:wandb.run_manager:file/dir modified: /content/drive/My Drive/Colab Notebooks/binary_classifier/wandb/run-20200608_152655-1r1hqqdt/wandb-history.jsonl
INFO:wandb.run_manager:file/dir m

INFO:simpletransformers.classification.classification_model:{'mcc': 0.9128717482439989, 'tp': 88, 'tn': 68, 'fp': 4, 'fn': 3, 'acc': 0.9570552147239264, 'f1_score': 0.9617486338797815, 'eval_loss': 0.22090506364613594}


INFO:wandb.run_manager:file/dir modified: /content/drive/My Drive/Colab Notebooks/binary_classifier/wandb/run-20200608_152655-1r1hqqdt/wandb-metadata.json
INFO:wandb.run_manager:file/dir modified: /content/drive/My Drive/Colab Notebooks/binary_classifier/wandb/run-20200608_152655-1r1hqqdt/wandb-history.jsonl
INFO:wandb.run_manager:file/dir modified: /content/drive/My Drive/Colab Notebooks/binary_classifier/wandb/run-20200608_152655-1r1hqqdt/wandb-summary.json
INFO:simpletransformers.classification.classification_model: Training of roberta model complete. Saved to outputs/.
INFO:wandb.run_manager:shutting down system stats and metadata service
INFO:wandb.run_manager:file/dir modified: /content/drive/My Drive/Colab Notebooks/binary_classifier/wandb/run-20200608_152655-1r1hqqdt/wandb-events.jsonl
INFO:wandb.run_manager:file/dir modified: /content/drive/My Drive/Colab Notebooks/binary_classifier/wandb/run-20200608_152655-1r1hqqdt/wandb-metadata.json
INFO:wandb.run_manager:stopping streamin

5. Evaluate the model

In [9]:
# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(eval_df, 
                                                            acc = sklearn.metrics.accuracy_score,
                                                            f1_score = sklearn.metrics.f1_score)

print(result)

INFO:simpletransformers.classification.classification_model: Features loaded from cache at cache_dir/cached_dev_roberta_128_2_163


INFO:simpletransformers.classification.classification_model:{'mcc': 0.9128717482439989, 'tp': 88, 'tn': 68, 'fp': 4, 'fn': 3, 'acc': 0.9570552147239264, 'f1_score': 0.9617486338797815, 'eval_loss': 0.22090506364613594}



{'mcc': 0.9128717482439989, 'tp': 88, 'tn': 68, 'fp': 4, 'fn': 3, 'acc': 0.9570552147239264, 'f1_score': 0.9617486338797815, 'eval_loss': 0.22090506364613594}


6. Loading Saved Models

In [0]:
binary_classifier = ClassificationModel('roberta', 
                                        'outputs/', 
                                        args=train_args)

7. Prediction

In [11]:
binary_pred, _ = binary_classifier.predict([
    'Aggressive disease containment efforts, including isolation of the source of infection, contact tracing and quarantine, social distancing, and personal protection and prevention, have considerably changed the course of Covid-19 outbreak in Wuhan, when there was neither effective drug nor vaccine for this new infectious disease with high transmission.',
    'If the infected are quarantined, cities will not accept more cases.',
    'Aggressive disease containment efforts, including isolation of the source of infection, contact tracing and quarantine, social distancing, and personal protection and prevention, have considerably changed the course of Covid-19 outbreak in Wuhan, when there was neither effective drug nor vaccine for this new infectious disease with high transmission.',
    'Shutting down public schools in New York City, for example, would deprive tens of thousands of kids of urgently needed school meals.',
    'Experts advise people to use hand sanitizers instead of masks because they can take up scarce medical resources.',
    'Experiments have shown that frequent hand washing does not reduce the spread of the virus from person to person as effectively as wearing a mask.',
    'In Singapore, for example, the government quickly canceled public events while private companies handed out free hand sanitizer. As a result, the number of cases has grown much more slowly than in nearby countries.',
    'And overall, the ban “non-essential” mass gatherings makes sense from a public health perspective.',
    'Without the intervention of contact reductions, in the longer term, the epidemics would increase exponentially. ',
    'Closely watching these contacts after exposure to an infected person will help the contacts to get care and treatment, and will prevent further transmission of the virus.',
    'Timely bans on public gatherings and closure of public places, including theaters and churches, were suggested to have had a positive effect on reducing the excess death rate during the 1918 pandemic in the United States (5,48). ',
    'Telecommuting, shift work and extended leave are some of the things that are not thought to mitigate the pandemic.',
    'Amongst the set of control measures tested, refraining from social activities with various compliance levels was relatively ineffective. ',
    'This increase in use of face masks by the general public exacerbates the global supply shortage of face masks, with prices soaring and risks supply constraints to frontline health-care professionals.',
    'This paper talked about isolation of the source of infection, contact tracing and quarantine, social distancing, and personal protection and prevention.',
    'The government issue the social distancing and wearing masks measures.',
    'Of importance, a variety of NPIs are contemporaneously in practice across countries in all continents, including lockdown strategies such as closure of schools, social distancing, quarantine, and isolation.'
])
print(binary_pred)

INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.
INFO:simpletransformers.classification.classification_model: Sliding window enabled


INFO:simpletransformers.classification.classification_model: 17 features created from 17 samples.



[1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 0 0]
